# Net Load Name Test
### 여기서는 Net 을 기존에 구성하여 저장한 뒤에, Variable 의 name 이 반드시 정확히 일치해야만 로드가 정확하게 들어가는지를 테스트 할 것 이다.

### 메타 데이터 로드하지 않고, 넷을 구성하되 다른 이름으로 구성을 한뒤, 안의 내용물이 동일하게 들어가는지를 확인할 것 이다.

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)
train_instance_num, pixel_num = mnist.train.images.shape
test_instance_num, class_num = mnist.test.labels.shape

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [4]:
x = tf.placeholder('float', shape=[None, pixel_num], name='x_data')
y_answer = tf.placeholder('float', shape=[None, class_num], name='y_labels')

In [5]:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # 입력데이터를 원래 이미지의 구조로 재구성한다.
x_image

<tf.Tensor 'Reshape:0' shape=(?, 28, 28, 1) dtype=float32>

In [6]:
conv1 = tf.layers.conv2d(inputs=x_image,
                filters=32,
                kernel_size=[3,3],
                padding='same',
                activation=tf.nn.relu,
                trainable=False,
                name='convolution_1')
conv1

<tf.Tensor 'convolution_1/Relu:0' shape=(?, 28, 28, 32) dtype=float32>

In [7]:
pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)
pool1

<tf.Tensor 'max_pooling2d/MaxPool:0' shape=(?, 14, 14, 32) dtype=float32>

In [8]:
conv2 = tf.layers.conv2d(inputs=pool1, 
                         filters=64, 
                         kernel_size=[3,3], 
                         padding='same', 
                         activation=tf.nn.relu, 
                         trainable=True, 
                         name='convolution_2')
conv2

<tf.Tensor 'convolution_2/Relu:0' shape=(?, 14, 14, 64) dtype=float32>

In [9]:
pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)
pool2

<tf.Tensor 'max_pooling2d_2/MaxPool:0' shape=(?, 7, 7, 64) dtype=float32>

In [10]:
pool2_flat = tf.reshape(pool2, [-1, pool2.shape[1].value*pool2.shape[2].value*pool2.shape[3].value])
pool2_flat

<tf.Tensor 'Reshape_1:0' shape=(?, 3136) dtype=float32>

In [11]:
dense1 = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
dense1

<tf.Tensor 'dense/Relu:0' shape=(?, 1024) dtype=float32>

In [12]:
dense2 = tf.layers.dense(inputs=dense1, units=10, activation=tf.nn.softmax)
dense2

<tf.Tensor 'dense_2/Softmax:0' shape=(?, 10) dtype=float32>

In [13]:
cross_entropy = - tf.reduce_sum(y_answer*tf.log(dense2)) # 크로스 엔트로피 방식으로 오차값을 선정
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(dense2, 1), tf.argmax(y_answer, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

In [14]:
saver = tf.train.Saver()
saver

In [15]:
[n.name for n in tf.get_default_graph().as_graph_def().node]

['init',
 'x_data',
 'y_labels',
 'Reshape/shape',
 'Reshape',
 'convolution_1/kernel/Initializer/random_uniform/shape',
 'convolution_1/kernel/Initializer/random_uniform/min',
 'convolution_1/kernel/Initializer/random_uniform/max',
 'convolution_1/kernel/Initializer/random_uniform/RandomUniform',
 'convolution_1/kernel/Initializer/random_uniform/sub',
 'convolution_1/kernel/Initializer/random_uniform/mul',
 'convolution_1/kernel/Initializer/random_uniform',
 'convolution_1/kernel',
 'convolution_1/kernel/Assign',
 'convolution_1/kernel/read',
 'convolution_1/bias/Initializer/Const',
 'convolution_1/bias',
 'convolution_1/bias/Assign',
 'convolution_1/bias/read',
 'convolution_1/convolution/Shape',
 'convolution_1/convolution/dilation_rate',
 'convolution_1/convolution',
 'convolution_1/BiasAdd',
 'convolution_1/Relu',
 'max_pooling2d/MaxPool',
 'convolution_2/kernel/Initializer/random_uniform/shape',
 'convolution_2/kernel/Initializer/random_uniform/min',
 'convolution_2/kernel/Initia

In [16]:
saver.restore(sess, 'mnist_ckpt/total-layers-model')

INFO:tensorflow:Restoring parameters from mnist_ckpt/total-layers-model


NotFoundError: Key convolution_2/bias/Adam not found in checkpoint
	 [[Node: save/RestoreV2_5 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save/Const_0, save/RestoreV2_5/tensor_names, save/RestoreV2_5/shape_and_slices)]]
	 [[Node: save/RestoreV2_3/_43 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_180_save/RestoreV2_3", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'save/RestoreV2_5', defined at:
  File "C:\Users\SoMa\Anaconda3\envs\venvJupyter\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\SoMa\Anaconda3\envs\venvJupyter\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\SoMa\Anaconda3\envs\venvJupyter\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
 

## 결론 : 저장(Save) 할때의 이름을 그대로 써주지 않으면, 순서대로는 동작하지 않는다.